In [1]:
import numpy as np
import pandas as pd
import os
import torch
from pathlib import Path
from torchvision.io import read_image
from torchvision.utils import make_grid
from torch.utils.data import Dataset, DataLoader, random_split
import matplotlib.pyplot as plt
from torchvision import models, transforms
import torch.optim as optim
import cv2

In [2]:
class ImageDataset(Dataset):
    def __init__(self, csv_path, dataset_dir, train, test, transform=None, target_transform=None):
        
        self.img_csv = pd.read_csv(csv_path, usecols = ["ImgPath", "Distance"])
        self.dataset_dir = dataset_dir
        self.transform = transforms.Resize([128, 128])
        self.target_transform = (1./15.)


    def __len__(self):
        return len(self.img_csv)

    def __getitem__(self, idx):
        img_path = os.path.join(self.dataset_dir, self.img_csv.iloc[idx, 1])
        image = read_image(img_path)[:3,:,:]
        label = self.img_csv.iloc[idx, 0]
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = label*self.target_transform
        return image, label

In [3]:
# image_dir = ""
# csv_path = "output/drone_dataset_28072022011417/data/drone.csv"

In [4]:
dataset_dir = "dataset/soccer_ball_07082022112157/"
csv_path = dataset_dir + "data/soccer_ball.csv"

In [5]:
# labels = pd.read_csv(csv_path, usecols = ["ImgPath", "Distance"]);

In [6]:
dataset = ImageDataset(csv_path, dataset_dir, train=True, test=False)
trainset,validset,testset = random_split(dataset,
                                         [int(len(dataset)*.65),int(len(dataset)*.15),int(len(dataset)*.20)],
                                         generator=torch.Generator().manual_seed(40))

trainloader = DataLoader(trainset, batch_size=32, shuffle=True)
validloader = DataLoader(validset, batch_size=32, shuffle=True)
testloader = DataLoader(testset, batch_size=32, shuffle=True)

In [7]:
print(len(trainset))
print(trainset[0][0].size())
print(trainset[0][1])

1300
torch.Size([3, 128, 128])
0.6289320754704399


In [8]:
import torch
import torch.nn as nn
from torch.nn import functional

class Model(nn.Module):
    def __init__(self, freeze_resnet = False):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 2, 3, stride=1, padding=0)
        self.conv2 = nn.Conv2d(2, 1, 3, stride=1, padding=0)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(900, 300)
        self.fc2 = nn.Linear(300, 30)
        self.fc3 = nn.Linear(30, 1)

    def forward(self, x):
        x = self.pool(functional.relu(self.conv1(x)))
        x = self.pool(functional.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = functional.relu(self.fc1(x))
        x = functional.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [9]:
def validation(model, validloader, criterion):
    
    val_loss = 0
    accuracy = 0
    
    for images, labels in iter(validloader):

#         images, labels = images.to('cuda'), labels.to('cuda')

        output = model.forward(images.type(torch.float))
        val_loss += criterion(output.type(torch.float), labels[:, None].type(torch.float)).item()
    
    return val_loss

In [10]:
# def train_classifier():
    
#     epochs = 100
#     steps = 0
#     print_every = 16
#     model_save_loc = "model/soccer_ball_07082022112157.pth"

# #     model.to('cuda')

#     for e in range(epochs):

#         model.train()

#         running_loss = 0

#         for images, labels in iter(trainloader):

#             steps += 1

# #             images, labels = images.to('cuda'), labels.to('cuda')

#             optimizer.zero_grad()
#             output = model.forward(images.float())
#             loss = criterion(output.float(), labels[:, None].float())
#             loss.backward()
#             optimizer.step()

#             running_loss += loss.item()

#             if steps % print_every == 0:

#                 model.eval()

#                 # Turn off gradients for validation, saves memory and computations
#                 with torch.no_grad():
#                     validation_loss = validation(model, validloader, criterion)

#                 print("Epoch: {}/{}.. ".format(e+1, epochs),
#                       "Training Loss: {:.3f}.. ".format(running_loss/print_every),
#                       "Validation Loss: {:.3f}.. ".format(validation_loss/len(validloader)))

#                 running_loss = 0
                
#                 torch.save(model, model_save_loc)
#                 model.train()

In [11]:
def train_classifier():
    
    epochs = 100
    last_loss = 10000
    patience = 3
    triggertimes = 0
    model_save_loc = "model/soccer_ball_07082022112157.pth"

#     model.to('cuda')

    for e in range(epochs):

        model.train()

        running_loss = 0

        for images, labels in iter(trainloader):

#             images, labels = images.to('cuda'), labels.to('cuda')

            optimizer.zero_grad()
            output = model.forward(images.float())
            loss = criterion(output.float(), labels[:, None].float())
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        model.eval()

        # Turn off gradients for validation, saves memory and computations
        with torch.no_grad():
            validation_loss = validation(model, validloader, criterion)

        print("Epoch: {}/{}.. ".format(e+1, epochs),
              "Training Loss: {:.3f}.. ".format(running_loss/len(trainloader)),
              "Validation Loss: {:.3f}.. ".format(validation_loss/len(validloader)))
        
        current_loss = validation_loss
        print('The Current Loss:', current_loss)
        
        if current_loss > last_loss:
            trigger_times += 1
            print('Trigger Times:', trigger_times, '\n')
            
            if trigger_times >= patience:
                print('Early stopping!\nStart to test process.')
                return model
            
        else:
            print('trigger times: 0', '\n')
            trigger_times = 0
            torch.save(model, model_save_loc)
            
        last_loss = current_loss
        
        model.train()
    return model

In [12]:
model = models.resnet18(weights = 'DEFAULT')
model.fc = nn.Linear(in_features=512, out_features=1)

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

res_model = train_classifier()

Epoch: 1/100..  Training Loss: 0.512..  Validation Loss: 0.039.. 
The Current Loss: 0.3864260297268629
trigger times: 0 

Epoch: 2/100..  Training Loss: 0.011..  Validation Loss: 0.007.. 
The Current Loss: 0.06986568868160248
trigger times: 0 

Epoch: 3/100..  Training Loss: 0.008..  Validation Loss: 0.005.. 
The Current Loss: 0.048398497980087996
trigger times: 0 

Epoch: 4/100..  Training Loss: 0.009..  Validation Loss: 0.010.. 
The Current Loss: 0.09918448724783957
Trigger Times: 1 

Epoch: 5/100..  Training Loss: 0.008..  Validation Loss: 0.003.. 
The Current Loss: 0.02562899561598897
trigger times: 0 

Epoch: 6/100..  Training Loss: 0.005..  Validation Loss: 0.002.. 
The Current Loss: 0.02494082774501294
trigger times: 0 

Epoch: 7/100..  Training Loss: 0.004..  Validation Loss: 0.003.. 
The Current Loss: 0.027670504874549806
Trigger Times: 1 

Epoch: 8/100..  Training Loss: 0.004..  Validation Loss: 0.003.. 
The Current Loss: 0.03300269087776542
Trigger Times: 2 

Epoch: 9/100.. 

In [19]:
def test_accuracy(model, testloader):
    
    model.eval()
#     model.to('cuda')
    loss = 0

    with torch.no_grad():
    
        for images, labels in iter(testloader):
    
#             images, labels = images.to('cuda'), labels.to('cuda')
    
            output = model.forward(images.type(torch.float))
            loss += criterion(output.type(torch.float), labels[:, None].type(torch.float)).item()
        
        print("\nTest Loss: {}".format(loss))
        print("Average error in predicted distance: {}".format(loss*15))

In [20]:
test_model = torch.load("model/soccer_ball_07082022112157.pth")
test_accuracy(test_model, testloader)


Test Loss: 0.018067445082124323
Average error in predicted distance: 0.27101167623186484
